# CCSD theory for a closed-shell reference

In this notebook we will use wicked to generate and implement equations for the CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers.py`.
In this example, we run a CCSD computation on the H<sub>6</sub> molecule, reading all the relevant information from the file `sr-h2o-cc-pvdz-spinorbital.npy`.

In [1]:
import time
import wicked as w
import numpy as np
from examples_helpers import *
import copy

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h2o-cc-pvdz-spinorbital.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [2]:
molecule = "sr-h2o-sto-3g-spinorbital"

with open(f"{molecule}.npy", "rb") as f:
    Eref = np.load(f)
    Ecorr_ref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()

invD = compute_inverse_denominators(H, nocc, nvir, 2)

## Compute the MP2 energy

To verify that the Hamiltonian is read correctly, we compute the MP2 correlation energy

In [3]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 * invD["oovv"][i][j][a][b]

print(f"MP2 correlation energy: {Emp2:.12f} Eh")

MP2 correlation energy: -0.049149644668 Eh


## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

In [4]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o"], unique=False)
Hop = w.utils.gen_op("H", 1, "ov", "ov") + w.utils.gen_op("H", 2, "ov", "ov")
# the similarity-transformed Hamiltonian truncated to the four-nested commutator term
Hbar = w.bch_series(Hop, Top, 4)

In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [5]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [6]:
energy_eq = generate_equation(mbeq, 0, 0)
t1_eq = generate_equation(mbeq, 1, 1)
t2_eq = generate_equation(mbeq, 2, 2)

exec(energy_eq)
exec(t1_eq)
exec(t2_eq)

# show what do these functions look like
print(energy_eq)

def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"],optimize="optimal")
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"],optimize="optimal")
    R += 0.500000000 * np.einsum("abij,jb,ia->",H["vvoo"],T["ov"],T["ov"],optimize="optimal")
    return R


## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [7]:
T = {"ov": np.zeros((nocc, nvir)), "oovv": np.zeros((nocc, nocc, nvir, nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 50

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_0_0(H, T)
    Etot_w = Eref + Ecorr_w
    R["ov"] = evaluate_residual_1_1(H, T)
    Roovv = evaluate_residual_2_2(H, T)
    R["oovv"] = antisymmetrize_residual_2_2(Roovv, nocc, nvir)

    # 2. amplitude update
    update_cc_amplitudes(T, R, invD, 2)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R["ov"]) ** 2 + np.linalg.norm(R["oovv"]) ** 2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}")
    if norm_R < 1.0e-8:
        break

end = time.perf_counter()
t = end - start

print("-" * len(header))
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:.4f} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)

-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -74.942079898873    +0.000000000000    6.792487e-01
  1    -74.991229543542    -0.049149644668    2.043067e-01
  2    -75.004838116006    -0.062758217133    8.209712e-02
  3    -75.009476493786    -0.067396594913    3.663817e-02
  4    -75.011304448168    -0.069224549294    1.718501e-02
  5    -75.012087669607    -0.070007770733    8.404825e-03
  6    -75.012439954105    -0.070360055231    4.270144e-03
  7    -75.012603732501    -0.070523833627    2.239147e-03
  8    -75.012681944941    -0.070602046068    1.202385e-03
  9    -75.012720205373    -0.070640306500    6.564612e-04
 10    -75.012739341188    -0.070659442314    3.623852e-04
 11    -75.012749106792    -0.070669207918    2.014838e-04
 12    -75.012754180417    -0.070674281544    1.125382e-04
 13    -75.012756857366    -0.070676958492    6.30418

### Build $\bar{H}$

In [8]:
for key in mbeq.keys():
    if key=='|': continue
    func = generate_tensor_block(mbeq[key], key)
    exec(func)

In [9]:
Hbar = {}
Hbar_oooo = evaluate_residual_oooo(H,T)
Hbar['oooo'] = antisymmetrize_residual_2_2_general(Hbar_oooo, 2, 2, 2, 2)
Hbar_ooov = evaluate_residual_ooov(H,T)
Hbar['ooov'] = antisymmetrize_residual_2_2_general(Hbar_ooov, 2, 2, 2, 1)
Hbar_oovv = evaluate_residual_oovv(H,T)
Hbar['oovv'] = antisymmetrize_residual_2_2_general(Hbar_oovv, 2, 2, 2, 2)
Hbar_ovoo = evaluate_residual_ovoo(H,T)
Hbar['ovoo'] = antisymmetrize_residual_2_2_general(Hbar_ovoo, 2, 2, 1, 2)
Hbar_ovov = evaluate_residual_ovov(H,T)
Hbar['ovov'] = antisymmetrize_residual_2_2_general(Hbar_ovov, 2, 2, 1, 1)
Hbar_ovvv = evaluate_residual_ovvv(H,T)
Hbar['ovvv'] = antisymmetrize_residual_2_2_general(Hbar_ovvv, 2, 2, 1, 2)
Hbar['oo'] = evaluate_residual_oo(H,T)
Hbar['ov'] = evaluate_residual_ov(H,T)
Hbar_vvoo = evaluate_residual_vvoo(H,T)
Hbar['vvoo'] = antisymmetrize_residual_2_2_general(Hbar_vvoo, 2, 2, 2, 2)
Hbar_vvov = evaluate_residual_vvov(H,T)
Hbar['vvov'] = antisymmetrize_residual_2_2_general(Hbar_vvov, 2, 2, 2, 1)
Hbar_vvvv = evaluate_residual_vvvv(H,T)
Hbar['vvvv'] = antisymmetrize_residual_2_2_general(Hbar_vvvv, 2, 2, 2, 2)
Hbar['vo'] = evaluate_residual_vo(H,T)
Hbar['vv'] = evaluate_residual_vv(H,T)

### Generate the 'Slater rules' for EOM

In [16]:
sd = ['v+ o','v+ v+ o o']
Hbar_op = Hop
THT = w.op('bra',sd).adjoint() @ Hbar_op @ w.op('c',sd)
expr = wt.contract(THT, 0, 0)
mbeq = expr.to_manybody_equation('sigma')

In [11]:
for eq in mbeq['|']:
    print(eq.compile('einsum'))

H += -1.000000000 * np.einsum("ji,aj,ia->",H["oo"],bra["vo"],sigma["ov"],optimize="optimal")
H += -0.500000000 * np.einsum("ji,abjk,ikab->",H["oo"],bra["vvoo"],sigma["oovv"],optimize="optimal")
H += 1.000000000 * np.einsum("ai,bj,ijab->",H["vo"],bra["vo"],sigma["oovv"],optimize="optimal")
H += 0.125000000 * np.einsum("klij,abkl,ijab->",H["oooo"],bra["vvoo"],sigma["oovv"],optimize="optimal")
H += 0.500000000 * np.einsum("kaij,bk,ijab->",H["ovoo"],bra["vo"],sigma["oovv"],optimize="optimal")
H += 0.500000000 * np.einsum("jkia,abjk,ib->",H["ooov"],bra["vvoo"],sigma["ov"],optimize="optimal")
H += -1.000000000 * np.einsum("jbia,aj,ib->",H["ovov"],bra["vo"],sigma["ov"],optimize="optimal")
H += -1.000000000 * np.einsum("jbia,acjk,ikbc->",H["ovov"],bra["vvoo"],sigma["oovv"],optimize="optimal")
H += 0.500000000 * np.einsum("bcia,aj,ijbc->",H["vvov"],bra["vo"],sigma["oovv"],optimize="optimal")
H += 1.000000000 * np.einsum("ia,abij,jb->",H["ov"],bra["vvoo"],sigma["ov"],optimize="optimal")
H += 1.0

In [12]:
sigma = {}
sigma['s'] = np.zeros((nocc, nvir))
sigma['d'] = np.zeros((nocc, nocc, nvir, nvir))
sigma['s'][-1,-1] = 1

In [17]:
for eq in mbeq['|']:
    print(w.compile_sigma_vector(eq))

sigma["vo"] += -1.00000000 * np.einsum('ij,ja->ai', H["oo"],c["ov"],optimize='optimal')
sigma["vvoo"] += -0.50000000 * np.einsum('ij,jkab->abik', H["oo"],c["oovv"],optimize='optimal')
sigma["vo"] += +1.00000000 * np.einsum('ai,ijab->bj', H["vo"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +0.12500000 * np.einsum('ijkl,klab->abij', H["oooo"],c["oovv"],optimize='optimal')
sigma["vo"] += +0.50000000 * np.einsum('iajk,jkab->bi', H["ovoo"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +0.50000000 * np.einsum('ijka,kb->abij', H["ooov"],c["ov"],optimize='optimal')
sigma["vo"] += -1.00000000 * np.einsum('iajb,ja->bi', H["ovov"],c["ov"],optimize='optimal')
sigma["vvoo"] += -1.00000000 * np.einsum('iajb,jkac->bcik', H["ovov"],c["oovv"],optimize='optimal')
sigma["vo"] += +0.50000000 * np.einsum('abic,ijab->cj', H["vvov"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +1.00000000 * np.einsum('ia,jb->abij', H["ov"],c["ov"],optimize='optimal')
sigma["vo"] += +1.00000000 * np.einsum('ab,ia->bi', 

### (Optional) Integral generation
For completeness, we document here how `sr-h2o-cc-pvdz-spinorbital.npy` was generated. You will need to install PySCF to run the following cell. Due to the size, we do not provide the cc-pVTZ integrals in the repository, but you can generate it with the cell below and see the difference to the spin-integrated code in the other notebook.

In [67]:
import pyscf, pyscf.cc, pyscf.mp

mol = pyscf.gto.M(atom="""
O 
H 1 1.1
H 1 1.1 2 104
""", basis='sto-3g')

mf = pyscf.scf.RHF(mol)
_ = mf.kernel()
cc = pyscf.cc.CCSD(mf)
_ = cc.kernel()
mp = pyscf.mp.MP2(mf)
_ = mp.kernel()

nocc = mol.nelectron
nvir = (mol.nao*2 - nocc)

eri = pyscf.ao2mo.full(mol.intor('int2e'), mf.mo_coeff)
eri = eri.swapaxes(1, 2)

blocks = get_index_blocks(energy_eq+t1_eq+t2_eq)

V = np.zeros((mol.nao*2, mol.nao*2, mol.nao*2, mol.nao*2))
V[::2,::2,::2,::2] = V[1::2,1::2,1::2,1::2] = eri - eri.swapaxes(2,3) # <aa||aa> and <bb||bb>
V[::2,1::2,::2,1::2] = V[1::2,::2,1::2,::2] = eri # <ab||ab> = <ba||ba> = <ab|ab>
V[::2,1::2,1::2,::2] = V[1::2,::2,::2,1::2] = -eri.swapaxes(2,3) # <ab||ba> = <ba||ab> = -<ab|ab>
F_spatorb = np.diag(mf.mo_energy)
F = np.zeros((mol.nao*2, mol.nao*2))
F[::2,::2] = F[1::2,1::2] = F_spatorb
sl = {'o': slice(0, nocc), 'v': slice(nocc, nocc+nvir)}
H = {}
for block in blocks:
    if len(block) == 2:
        H[block] = F[sl[block[0]], sl[block[1]]]
    else:
        H[block] = V[sl[block[0]], sl[block[1]], sl[block[2]], sl[block[3]]]

with open('sr-h2o-sto-3g-spinorbital.npy', 'wb') as f:
    np.save(f, mf.e_tot)
    np.save(f, cc.e_corr)
    np.save(f, (nocc,nvir))
    np.save(f, H, allow_pickle=True)

converged SCF energy = -74.9420798988735
E(CCSD) = -75.01275999574094  E_corr = -0.07068009686744721
E(MP2) = -74.9912295435419  E_corr = -0.0491496446683854
E(SCS-MP2) = -74.9983674136603  E_corr = -0.0562875147868231
